# Olist E-Commerce Data

**Setting up Spark Environment**

1\. Deploy a spark cluster (Dataproc, EMR, HDInsight, On-Premis)

2\. Download the data from Kaggle website [Link](https://www.kaggle.com/api/v1/datasets/download/olistbr/brazilian-ecommerce)

3\. Bash command to direct download to HDFS 
- Make sure to create a proper directory to put downloaded and to extract files into
- I created olist folder to download file
```
curl -L -o ~/olist/brazilian-ecommerce.zip\
  https://www.kaggle.com/api/v1/datasets/download/olistbr/brazilian-ecommerce
```
- I created data folder inside olist to extract files

```
unzip brazilian-ecommerce.zip -d ~/olist/data/
```

4\. Store Data in HDFS worker nodes
- I create folder to distribute files from hadoop master node to worker nodes
```
!hadoop fs -put ~/olist/data/*.csv /data/olist/
```
    
5\. Use Pyspark to intract with Data



# Data Ingestion

**Downloaded and extracted data in hadoop master cluster**

In [21]:
ls /home/sekar_dhana8644/olist/

brazilian-ecommerce.zip  data/


In [20]:
ls /home/sekar_dhana8644/olist/data/

olist_customers_dataset.csv       olist_orders_dataset.csv
olist_geolocation_dataset.csv     olist_products_dataset.csv
olist_order_items_dataset.csv     olist_sellers_dataset.csv
olist_order_payments_dataset.csv  product_category_name_translation.csv
olist_order_reviews_dataset.csv


**Moved data from Hadoop master cluster to worker nodes**

In [19]:
!hadoop fs -ls -h /data/olist/

Found 9 items
-rw-r--r--   2 sekar_dhana8644 hadoop      8.6 M 2025-08-30 19:17 /data/olist/olist_customers_dataset.csv
-rw-r--r--   2 sekar_dhana8644 hadoop     58.4 M 2025-08-30 19:17 /data/olist/olist_geolocation_dataset.csv
-rw-r--r--   2 sekar_dhana8644 hadoop     14.7 M 2025-08-30 19:17 /data/olist/olist_order_items_dataset.csv
-rw-r--r--   2 sekar_dhana8644 hadoop      5.5 M 2025-08-30 19:17 /data/olist/olist_order_payments_dataset.csv
-rw-r--r--   2 sekar_dhana8644 hadoop     13.8 M 2025-08-30 19:17 /data/olist/olist_order_reviews_dataset.csv
-rw-r--r--   2 sekar_dhana8644 hadoop     16.8 M 2025-08-30 19:17 /data/olist/olist_orders_dataset.csv
-rw-r--r--   2 sekar_dhana8644 hadoop      2.3 M 2025-08-30 19:17 /data/olist/olist_products_dataset.csv
-rw-r--r--   2 sekar_dhana8644 hadoop    170.6 K 2025-08-30 19:17 /data/olist/olist_sellers_dataset.csv
-rw-r--r--   2 sekar_dhana8644 hadoop      2.6 K 2025-08-30 19:17 /data/olist/product_category_name_translation.csv


# Data Exploration

In [2]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName('OlistDataset').getOrCreate()

spark

25/08/31 18:17:50 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


In [3]:
hdfs_path = '/data/olist/'

In [4]:
customer_df = spark.read.csv(hdfs_path + 'olist_customers_dataset.csv',header=True, inferSchema=True)

In [28]:
customer_df.show(5)

+--------------------+--------------------+------------------------+--------------------+--------------+
|         customer_id|  customer_unique_id|customer_zip_code_prefix|       customer_city|customer_state|
+--------------------+--------------------+------------------------+--------------------+--------------+
|06b8999e2fba1a1fb...|861eff4711a542e4b...|                   14409|              franca|            SP|
|18955e83d337fd6b2...|290c77bc529b7ac93...|                    9790|sao bernardo do c...|            SP|
|4e7b3e00288586ebd...|060e732b5b29e8181...|                    1151|           sao paulo|            SP|
|b2b6027bc5c5109e5...|259dac757896d24d7...|                    8775|     mogi das cruzes|            SP|
|4f2d8ab171c80ec83...|345ecd01c38d18a90...|                   13056|            campinas|            SP|
+--------------------+--------------------+------------------------+--------------------+--------------+
only showing top 5 rows



In [5]:
geolocation_df = spark.read.csv(hdfs_path + 'olist_geolocation_dataset.csv',header=True, inferSchema=True)
order_items_df = spark.read.csv(hdfs_path + 'olist_order_items_dataset.csv',header=True, inferSchema=True)
order_payments_df = spark.read.csv(hdfs_path + 'olist_order_payments_dataset.csv',header=True, inferSchema=True)
order_reviews_df = spark.read.csv(hdfs_path + 'olist_order_reviews_dataset.csv',header=True, inferSchema=True)
orders_df = spark.read.csv(hdfs_path + 'olist_orders_dataset.csv',header=True, inferSchema=True)
products_df = spark.read.csv(hdfs_path + 'olist_products_dataset.csv',header=True, inferSchema=True)
sellers_df = spark.read.csv(hdfs_path + 'olist_sellers_dataset.csv',header=True, inferSchema=True)
translation_df = spark.read.csv(hdfs_path + 'product_category_name_translation.csv',header=True, inferSchema=True)

In [43]:
customer_df.printSchema()

root
 |-- customer_id: string (nullable = true)
 |-- customer_unique_id: string (nullable = true)
 |-- customer_zip_code_prefix: integer (nullable = true)
 |-- customer_city: string (nullable = true)
 |-- customer_state: string (nullable = true)



In [45]:
orders_df.printSchema()

root
 |-- order_id: string (nullable = true)
 |-- customer_id: string (nullable = true)
 |-- order_status: string (nullable = true)
 |-- order_purchase_timestamp: timestamp (nullable = true)
 |-- order_approved_at: timestamp (nullable = true)
 |-- order_delivered_carrier_date: timestamp (nullable = true)
 |-- order_delivered_customer_date: timestamp (nullable = true)
 |-- order_estimated_delivery_date: timestamp (nullable = true)



In [6]:
# Data Leakage or Drop

print(f'Customers df: {customer_df.count()} rows')
print(f'Orders df: {orders_df.count()} rows')
print(f'Order Items df: {order_items_df.count()} rows')
print(f'Order Payments df: {order_payments_df.count()} rows')
print(f'Order Reviews df: {order_reviews_df.count()} rows')
print(f'Products df: {products_df.count()} rows')
print(f'Sellers df: {sellers_df.count()} rows')
print(f'Product Category Name Translation df: {translation_df.count()} rows')

Customers df: 99441 rows
Orders df: 99441 rows
Order Items df: 112650 rows
Order Payments df: 103886 rows
Order Reviews df: 104162 rows
Products df: 32951 rows
Sellers df: 3095 rows
Product Category Name Translation df: 71 rows


In [9]:
# Null Values

from pyspark.sql.functions import *

customer_df.select([count(when(col(c).isNull(),1)).alias(c) for c in customer_df.columns]).show()

+-----------+------------------+------------------------+-------------+--------------+
|customer_id|customer_unique_id|customer_zip_code_prefix|customer_city|customer_state|
+-----------+------------------+------------------------+-------------+--------------+
|          0|                 0|                       0|            0|             0|
+-----------+------------------+------------------------+-------------+--------------+



In [7]:
# Duplicate Values

customer_df.groupBy('customer_id').count().filter('count>1').show()

+-----------+-----+
|customer_id|count|
+-----------+-----+
+-----------+-----+



In [10]:
# Customer Distribution by State

customer_df.groupBy('customer_state').count().orderBy(col('count').desc()).show()

+--------------+-----+
|customer_state|count|
+--------------+-----+
|            SP|41746|
|            RJ|12852|
|            MG|11635|
|            RS| 5466|
|            PR| 5045|
|            SC| 3637|
|            BA| 3380|
|            DF| 2140|
|            ES| 2033|
|            GO| 2020|
|            PE| 1652|
|            CE| 1336|
|            PA|  975|
|            MT|  907|
|            MA|  747|
|            MS|  715|
|            PB|  536|
|            PI|  495|
|            RN|  485|
|            AL|  413|
+--------------+-----+
only showing top 20 rows



In [13]:
# Customer Distribution by City

customer_df.groupBy('customer_city').count().orderBy('count', ascending=False).show()

+--------------------+-----+
|       customer_city|count|
+--------------------+-----+
|           sao paulo|15540|
|      rio de janeiro| 6882|
|      belo horizonte| 2773|
|            brasilia| 2131|
|            curitiba| 1521|
|            campinas| 1444|
|        porto alegre| 1379|
|            salvador| 1245|
|           guarulhos| 1189|
|sao bernardo do c...|  938|
|             niteroi|  849|
|         santo andre|  797|
|              osasco|  746|
|              santos|  713|
|             goiania|  692|
| sao jose dos campos|  691|
|           fortaleza|  654|
|            sorocaba|  633|
|              recife|  613|
|       florianopolis|  570|
+--------------------+-----+
only showing top 20 rows



In [17]:
orders_df.columns

['order_id',
 'customer_id',
 'order_status',
 'order_purchase_timestamp',
 'order_approved_at',
 'order_delivered_carrier_date',
 'order_delivered_customer_date',
 'order_estimated_delivery_date']

In [18]:
# Order Distribution by Order Status

orders_df.groupBy('order_status').count().orderBy('count', ascending=False).show()

+------------+-----+
|order_status|count|
+------------+-----+
|   delivered|96478|
|     shipped| 1107|
|    canceled|  625|
| unavailable|  609|
|    invoiced|  314|
|  processing|  301|
|     created|    5|
|    approved|    2|
+------------+-----+



In [35]:
delivery_df = orders_df.select('order_id', 'order_purchase_timestamp', 'order_delivered_customer_date')
delivery_df.show(5)

+--------------------+------------------------+-----------------------------+
|            order_id|order_purchase_timestamp|order_delivered_customer_date|
+--------------------+------------------------+-----------------------------+
|e481f51cbdc54678b...|     2017-10-02 10:56:33|          2017-10-10 21:25:13|
|53cdb2fc8bc7dce0b...|     2018-07-24 20:41:37|          2018-08-07 15:27:45|
|47770eb9100c2d0c4...|     2018-08-08 08:38:49|          2018-08-17 18:06:29|
|949d5b44dbf5de918...|     2017-11-18 19:28:06|          2017-12-02 00:28:42|
|ad21c59c0840e6cb8...|     2018-02-13 21:18:39|          2018-02-16 18:17:02|
+--------------------+------------------------+-----------------------------+
only showing top 5 rows



In [36]:
delivery_time = delivery_df.withColumn('delivery_time', datediff(col('order_delivered_customer_date'), col('order_purchase_timestamp')))
delivery_time.show(5)

+--------------------+------------------------+-----------------------------+-------------+
|            order_id|order_purchase_timestamp|order_delivered_customer_date|delivery_time|
+--------------------+------------------------+-----------------------------+-------------+
|e481f51cbdc54678b...|     2017-10-02 10:56:33|          2017-10-10 21:25:13|            8|
|53cdb2fc8bc7dce0b...|     2018-07-24 20:41:37|          2018-08-07 15:27:45|           14|
|47770eb9100c2d0c4...|     2018-08-08 08:38:49|          2018-08-17 18:06:29|            9|
|949d5b44dbf5de918...|     2017-11-18 19:28:06|          2017-12-02 00:28:42|           14|
|ad21c59c0840e6cb8...|     2018-02-13 21:18:39|          2018-02-16 18:17:02|            3|
+--------------------+------------------------+-----------------------------+-------------+
only showing top 5 rows



In [37]:
# Average Order Delivery Time

delivery_time.orderBy('delivery_time', ascending=False).show(5)

+--------------------+------------------------+-----------------------------+-------------+
|            order_id|order_purchase_timestamp|order_delivered_customer_date|delivery_time|
+--------------------+------------------------+-----------------------------+-------------+
|ca07593549f1816d2...|     2017-02-21 23:31:27|          2017-09-19 14:36:39|          210|
|1b3190b2dfa9d789e...|     2018-02-23 14:57:35|          2018-09-19 23:24:07|          208|
|440d0d17af552815d...|     2017-03-07 23:59:51|          2017-09-19 15:12:50|          196|
|2fb597c2f772eca01...|     2017-03-08 18:09:02|          2017-09-19 14:33:17|          195|
|285ab9426d6982034...|     2017-03-08 22:47:40|          2017-09-19 14:00:04|          195|
+--------------------+------------------------+-----------------------------+-------------+
only showing top 5 rows



In [32]:
order_payments_df.columns

['order_id',
 'payment_sequential',
 'payment_type',
 'payment_installments',
 'payment_value']

In [20]:
# Payment Distribution by Payment Method

order_payments_df.groupBy('payment_type').count().orderBy('count', ascending=False).show()

+------------+-----+
|payment_type|count|
+------------+-----+
| credit_card|76795|
|      boleto|19784|
|     voucher| 5775|
|  debit_card| 1529|
| not_defined|    3|
+------------+-----+



In [21]:
order_items_df.columns

['order_id',
 'order_item_id',
 'product_id',
 'seller_id',
 'shipping_limit_date',
 'price',
 'freight_value']

In [26]:
# Top products by sales amount

top_product = order_items_df.groupBy('product_id').agg(sum('price').alias('total_sales'), count('product_id').alias('count'))
top_product.orderBy('total_sales', ascending=False).show()

+--------------------+------------------+-----+
|          product_id|       total_sales|count|
+--------------------+------------------+-----+
|bb50f2e236e5eea01...|           63885.0|  195|
|6cdd53843498f9289...| 54730.20000000005|  156|
|d6160fb7873f18409...|48899.340000000004|   35|
|d1c427060a0f73f6b...| 47214.51000000006|  343|
|99a4788cb24856965...|43025.560000000085|  488|
|3dd2a17168ec895c7...| 41082.60000000005|  274|
|25c38557cf793876c...| 38907.32000000001|   38|
|5f504b3a1c75b73d6...|37733.899999999994|   63|
|53b36df67ebb7c415...| 37683.42000000001|  323|
|aca2eb7d00ea1a7b8...| 37608.90000000007|  527|
|e0d64dcfaa3b6db5c...|          31786.82|  194|
|d285360f29ac7fd97...|31623.809999999983|  123|
|7a10781637204d8d1...|           30467.5|  143|
|f1c7f353075ce59d8...|          29997.36|  154|
|f819f0c84a64f02d3...|29024.479999999996|   45|
|588531f8ec37e7d5f...|28291.989999999998|   20|
|422879e10f4668299...|26577.219999999972|  484|
|16c4e87b98a9370a9...|           25034.0